In [1]:
import sys
sys.path.append("../../../experiments/") # use utils

import utils
import importlib
importlib.reload(utils)

from utils import calc_auc, get_abstract, normalize_text

import requests
import json
from progressbar import progressbar as pb
from utils import API_KEY, SEARCH_ENGINE_ID
from copy import deepcopy

In [9]:
with open("../../../data/doc_id2abstract.json") as f:
    doc_id2abstract = json.load(f)

In [2]:
api_key = API_KEY
search_engine_id = SEARCH_ENGINE_ID

url = f"https://www.googleapis.com/customsearch/v1?key={api_key}&cx={search_engine_id}&q=<QUERY>"

In [3]:
df = pd.read_csv("../../../data/data_to_process.csv")
df.head(3)

,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


## Keywords

### Pt. 1

In [6]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(df.iterrows(), max_value=len(df)):
    query_id = row.query_id
    data_source = row.data_source
    query = row.query
    
    new_url = deepcopy(url)
    new_url = new_url.replace("<QUERY>", query)
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    if "items" in response:
        for resp in response['items']:
            doc_id = resp['link'].split("/")[-2]
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)

100% (50 of 50) |########################| Elapsed Time: 0:00:35 Time:  0:00:35


In [4]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(df, on=["query_id", "data_source"])
)

In [5]:
fin_df['abstract'] = fin_df.pubmed_id.map(doc_id2abstract)

In [24]:
fin_df.to_csv("../../../data/google_search/google_search_abstracts_keywords_part_1.csv", index=0)

### Pt. 2

In [6]:
pt1 = pd.read_csv("../../../data/google_search/google_search_abstracts_keywords_part_1.csv")
pt2_to_search = (
    df
    .merge(pt1, how='left', indicator=True)
    .query("_merge == 'left_only'")
    .drop(["pubmed_id", 'abstract', '_merge'], axis=1)
    .reset_index(drop=True)
)

In [69]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(pt2_to_search.iterrows(), max_value=len(pt2_to_search)):
    query_id = row.query_id
    data_source = row.data_source
    query = row.query
    
    new_url = deepcopy(url)
    new_url = new_url.replace("<QUERY>", query)
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    if "items" in response:
        for resp in response['items']:
            doc_id = resp['link'].split("/")[-2]
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)

100% (15 of 15) |########################| Elapsed Time: 0:00:10 Time:  0:00:10


In [7]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(pt2_to_search, on=["query_id", "data_source"])
)

In [8]:
fin_df['abstract'] = fin_df.pubmed_id.map(doc_id2abstract)

In [86]:
fin_df.to_csv("../../../data/google_search/google_search_abstracts_keywords_part_2.csv", index=0)

## Question

### Pt. 1

In [9]:
df = pd.read_csv("../../../data/data_to_process.csv")
df.head(3)

,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [26]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(df.iterrows(), max_value=len(df)):
    query_id = row.query_id
    data_source = row.data_source
    query = normalize_text(row.description, False).strip()
    
    new_url = deepcopy(url)
    new_url = new_url.replace("<QUERY>", query)
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    if "items" in response:
        for resp in response['items']:
            doc_id = resp['link'].split("/")[-2]
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)

100% (50 of 50) |########################| Elapsed Time: 0:00:36 Time:  0:00:36


In [10]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(df, on=["query_id", "data_source"])
)

In [11]:
fin_df['abstract'] = fin_df.pubmed_id.map(doc_id2abstract)

In [37]:
fin_df.to_csv("../../../data/google_search/google_search_abstracts_question_part_1.csv", index=0)

### Pt. 2

In [12]:
df = pd.read_csv("../../../data/data_to_process.csv")
df.head(3)

,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [ ]:
pt1 = pd.read_csv("../../../data/google_search/google_search_abstracts_question_part_1.csv")
pt1 = pt1.astype({"query_id": object})
pt2_to_search = (
    df
    .merge(pt1, on=['query', 'description', 'data_source', 'label'], how='left', indicator=True)
    .query("_merge == 'left_only'")
    .drop(["pubmed_id", 'abstract', '_merge', 'query_id_y'], axis=1)
    .rename(columns={"query_id_x": "query_id"})
    .reset_index(drop=True)
)

In [146]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(pt2_to_search.iterrows(), max_value=len(pt2_to_search)):
    query_id = row.query_id
    data_source = row.data_source
    query = normalize_text(row.description, False).strip()
    
    new_url = deepcopy(url)
    new_url = new_url.replace("<QUERY>", query)
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    if "items" in response:
        for resp in response['items']:
            doc_id = resp['link'].split("/")[-2]
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)

100% (28 of 28) |########################| Elapsed Time: 0:00:16 Time:  0:00:16


In [ ]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(pt2_to_search, on=["query_id", "data_source"])
)

In [ ]:
fin_df['abstract'] = fin_df.pubmed_id.map(doc_id2abstract)

In [152]:
fin_df.to_csv("../../../data/google_search/google_search_abstracts_question_part_2.csv", index=0)